# Practice Session 04: Basket analysis

Association rule mining techniques are useful to analyze datasets consisting of transactions, in which each transaction is a collection of items.

We will use a well-known dataset named [Instacart](https://www.kaggle.com/c/instacart-market-basket-analysis) containing more than 3 million orders of products through a grocery shopping app. You can find it in the `instacart/` directory of the practicum data files.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

Author: <font color="blue">Your name here</font>

E-mail: <font color="blue">Your e-mail here</font>

Date: <font color="blue">The current date here</font>

In [1]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
import csv
import gzip
from apyori import apriori

If the apyori library is not already installed in your laptop, you can install it with: `pip install apyori`

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

## 0. The Apriori Algorithm in a nutshell

There are three major components of Apriori algorithm, which we describe below using as an example the case where transactions are purchase histories.

**Support**: the number of transactions containing a particular item divided by total number of transactions:

   *Support(A) = (Transactions containing (A))/(Total Transactions)*

**Confidence**: normally indicates the likelihood that an item B is also bought if item A is bought. It can be calculated by finding the number of transactions where A and B are bought together, divided by total number of transactions where A is bought:

   *Confidence(A→B) = (Transactions containing both (A and B))/(Transactions containing A)*

**Lift**: the increase in the ratio of sale of B when A is sold. Lift(A –> B) can be calculated by dividing Confidence(A -> B) by Support(B):

   *Lift(A→B) = (Confidence (A→B))/(Support (B))*
   
A Lift of 1 means there is no association between products A and B. Lift greater than 1.0 means products A and B are more likely to be bought together. Lift less than 1.0 indicates two products are unlikely to be bought together.

The Apriori algorithm first finds itemsets having the desired level of support, and then within those itemsets tries to derive rules having the desired confidence and lift.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

# 1. Playing with apyori

The [apyori library](https://pypi.org/project/apyori/) is an implementation of the Apriori algorithm. Its typical usage is to receive a list of transactions and then print the association rules it found.

To use this library, we pass a list in which each element represents a transaction, for instance:

```python
transactions = [
    ['beer', 'chips', 'nuts', 'olives'],
    ['beer', 'chips', 'olives'],
    ['chips', 'nuts' ],
    ['chips', 'olives'],
    ['beer', 'nuts' ],
    ['chips'],
    ['nuts', 'olives'],
    ['beer', 'nuts'],
    ['beer', 'chips', 'olives'], 
    ['beer', 'nuts', 'olives'], 

]
results = list(apriori(transactions, min_support=0.2, min_confidence=0.75, min_lift=1.0))

```

The function below, which you can leave as-is, prints the output of the apyori library in a readable format. Use it to print the results of your association rules mining:

```python
print_apyori_output(results)
```

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [2]:
# LEAVE AS-IS

def print_apyori_output (association_results, info=False, info_key=False):
    for relation_record in association_results:
        itemset = list(relation_record.items)
        
        # Consider only itemsets of two elements
        if len(itemset) > 1: 
        
            print("Rules involving itemset %s" % itemset)
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)
                
                if info_key:
                    antecedent = [info.loc[x][info_key] for x in antecedent]
                    consequent = [info.loc[x][info_key] for x in consequent]
                
                confidence = rules.confidence
                lift = rules.lift

                print("%s => %s (support=%.4f, confidence=%.2f, lift=%.2f)" %
                      (antecedent, consequent, support, confidence, lift))
            print()

<font size="+1" color="red">Replace this cell with your own example of transactions (at least 20 transactions) and execution of the apriori algorithm, in which you should obtain at least ONE and at most THREE rules.</font>

<font size="+1" color="red">Replace this cell with (1) a printout of the rules you have obtained, and (2) for each of those rules, indicate clearly how the support, confidence, and lift is calculated. Do not merely repeat the formula: indicate how each number is computed based on the transactions you provided, as if you were trying to verify that the numbers are correct.</font>

# 2. Load and prepare the shopping baskets

The following code, which you should leave as-is, loads the information about products into a dataframe indexed by product id.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [5]:
# LEAVE AS-IS

# File names
INPUT_PRODUCTS = "instacart-products.csv"
INPUT_TRANSACTIONS = "instacart-transactions.csv.gz"

# Read into a dataframe
products = pd.read_csv(INPUT_PRODUCTS, delimiter=",")

# Set product_id as index, and drop column aisle_id
products = products.set_index('product_id').drop(columns=['aisle_id'])

products.head(100)

,product_name,department_id
product_id,,
1,Chocolate Sandwich Cookies,19
2,All-Seasons Salt,13
3,Robust Golden Unsweetened Oolong Tea,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1
5,Green Chile Anytime Sauce,13
...,...,...
96,Sprinklez Confetti Fun Organic Toppings,13
97,Organic Chamomile Lemon Tea,7
98,2% Yellow American Cheese,16


## 2.1. Select by department

As this file is large and complex, we will focus on one or two departments and try to get some conclusions about the products in those departments. The following cell, which you should leave as-is, list some department names.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

In [6]:
# LEAVE AS-IS

DEPT_BAKERY = 3
DEPT_VEGGIES = 4
DEPT_ALCOHOL = 5
DEPT_WORLD = 6
DEPT_DRINKS = 7
DEPT_PETS = 8
DEPT_PHARMACY = 11
DEPT_CLEANING = 17
DEPT_BABIES = 18

Write code that can select a list of products from a set of departments. Do this with a function named `select_from_departments` that takes as input:

* A dataframe containing product information, which will be the `products` dataframe we just loaded.
* A list of product ids
* A list of department ids

It should return a list containing only the product ids that belong to one of the listed departments. This may return an empty list if no product belongs to any of the specified departments.

Given that the products dataframe is indexed by *product_id*, if you want to obtain the *department_id* of product *product_id*, use:

```python
products.loc[product_id].department_id
```

Note that *product_id* must be an integer.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code for *select_from_departments*.</font>

Test your function by passing it a list of products and ensuring it selects only the products in the 1-2 departments you have selected. To obtain test cases you can open the products file with a spreadsheet program.

Each test case should print:

* The product name and department id of each item in the input list
* The product name and department id of each item in the output list

For instance, suppose a test case is `[21, 26, 45, 54, 57, 71, 111, 112]` and we select products from DEPT_PETS and DEPT_CLEANING, a test case run should print something similar to this:

```
Test case:
[21, 26, 45, 54, 57, 71, 111, 112]

Input products:
21 Small & Medium Dental Dog Treats (dept 8)
26 Fancy Feast Trout Feast Flaked Wet Cat Food (dept 8)
45 European Cucumber (dept 4)
54 24/7 Performance Cat Litter (dept 8)
57 Flat Toothpicks (dept 17)
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17)
111 Fabric Softener, Geranium Scent (dept 17)
112 Hot Tomatillo Salsa (dept 13)

Selected products:
21 Small & Medium Dental Dog Treats (dept 8)
26 Fancy Feast Trout Feast Flaked Wet Cat Food (dept 8)
54 24/7 Performance Cat Litter (dept 8)
57 Flat Toothpicks (dept 17)
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17)
111 Fabric Softener, Geranium Scent (dept 17)
```

Do not replicate code that can be easily factored in a function in your answer.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with code to test your function with three different test cases. Each test case is a list of items and a list of 1, 2, or 3 departments.</font>

## 2.2. Read and filter transactions

The transactions file is a compressed file containing one row per transaction. Each transaction is a comma-separated list of *product_id*. The following skeleton iterates through this file:

```python
# Open a compressed file
with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:
    
    # Create a CSV reader
    reader = csv.reader(inputfile, delimiter=",")
    
    # Iterate through the CSV file
    for row in reader:
        
        # Convert to integers
        items = [int(x) for x in row]
```

Read the transactions, filtering the items so they belong to the DEPT_CLEANING department. Stop reading (`break`) after you have stored 5000 transactions into an array named `transactions`. Every 1000 transactions read, print the number of transactions read and the number of transactions stored.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to read transactions, keeping only items in DEPT_CLEANING. Remember to stop after keeping 5000 transactions.</font>

## 2.3. Extract association rules and comment on them (DEPT_CLEANING)

You are now ready to run the association rules mining algorithm over the selected transactions:

```python
results = list(apriori(transactions, min_support=..., min_confidence=..., min_lift=...))
print_apyori_output(results, products, 'product_name')
```

*Tip:* if you set `min_support` to a very small value, your notebook will probably hang.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+1" color="red">Replace this cell with your code to extract association rules from the read transactions (DEPT_CLEANING).</font>

<font size="+1" color="red">Replace this cell with a brief commentary on what you would recommend to the shopping app considering the extracted association rules.</font>

## 2.4. Extract association rules and comment on them (other departments)

<font size="+1" color="red">Replace this cell with code to select a different set of departments (at least two, not DEPT_CLEANING) and extract transactions again. Avoid replicating code when possible.</font>

<font size="+1" color="red">Replace this cell with your commentary on the obtained rules.</font>

# DELIVER (individually)

Remember to read the section on "delivering your code" in the [course evaluation guidelines](https://github.com/chatox/data-mining-course/blob/master/upf/upf-evaluation.md).

Deliver a zip file containing:

* This notebook

## Extra points available

For more learning and extra points, write code to filter the obtained association rules so that you print only the ones involving products in different departments. To be precise, this means rules in which there is at least a product in the *consequence* that belongs to a department that none of the products in the *antecedent* belongs to. Experiment with different combinations of departments, and try to discover interesting groups of products in different departments that are related to each other.

**Note:** if you go for the extra points, add ``<font size="+2" color="blue">Additional results: experiments on cross-department association rules</font>`` at the top of your notebook.

<font size="-1" color="gray">(Remove this cell when delivering.)</font>

<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>